In [2]:
import pymysql
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)

mysql_host = "142.132.134.21"
mysql_prot = 9030
mysql_user = "dw"
mysql_pd = "a64imrJwqKZc9jiU!*aCYi.xnh9*TVhhBmZi"
mysql_db = "dw"
connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db=mysql_db, port=mysql_prot, charset='utf8')
cursor = connection.cursor()
logging.info("connection created")


def getSqlResult(sql):
    cursor.execute(sql)
    res_rows = cursor.fetchall()
    df = pd.DataFrame(res_rows)
    return df

INFO:root:connection created


In [17]:
txn_hash = '0x958236266991bc3fe3b77feaacea120f172c0708ad01c7a715b255f218f9313c'
txn_detail_sql = """
select a.block_number,a.log_index,a.transaction_hash
    ,a.token_address,b.symbol,b.token_description,b.decimals
    ,a.from_address,c.tag_code
    ,a.to_address,c.tag_code
    ,a.value
    ,a.value/POWER(10, b.decimals) as txn_value
from prod.transfer_event_eth as a
left join prod.token as b
on a.token_address = b.address
left join
(
    select address
        ,max(tag_code) as tag_code
    from prod.tag_address
    group by 1
) as c
on a.from_address = c.address
left join 
(
    select address
        ,max(tag_code) as tag_code
    from prod.tag_address
    group by 1
) as d
on a.to_address = d.address
where a.transaction_hash = '{txn_hash}'
order by log_index asc
""".format(txn_hash=txn_hash)
df = pd.read_sql(txn_detail_sql, con=connection)

/opt/conda/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# Coinbase Transfer

In [4]:
def get_coinbase_trf(txn_list,start_block,end_block):
    sql ="""
    select a.block_number
        ,a.transaction_hash
        ,max(case when a.miner is not null then a.amount else 0 end) as coinbase_trf_amt
        ,max(b.effective_gas_price * b.gas_used / POWER(10.0, 18)) as gas_fee
    from (
        select a.block_number,a.transaction_hash
            ,a.from_address,a.to_address
            ,b.miner,a.token_address,a.symbol,a.amount
        from dw.dwb_token_transfer_detail_eth_hi as a
        left join prod.blocks as b
        on a.block_number = b.block_number
        and a.to_address = b.miner
        and b.block_number >= {start_block} and b.block_number <= {end_block}
        and b.`chain` = 'eth'
        where a.block_number >= {start_block} and a.block_number <= {end_block}
        and a.transaction_hash in ({txn_list})
    ) as a 
    left join prod.transactions_eth as b
    on a.transaction_hash = b.transaction_hash
    and b.block_number >= {start_block} and b.block_number<= {end_block}
    group by 1,2
    """.format(
        start_block = start_block,
        end_block = end_block,
        txn_list = ",".join('"'+txn+'"' for txn in txn_list)
    )
    df = getSqlResult(sql)
    return df

txn_list = [
    '0xf41775f2a03045511deee450845d6e197a2909ac46c2f3759f2dff699eee5aa1',
    '0xec7b3c821492dd76ac274b7ac37e0b55cabe535ecbd917b740f8d3389c2c3b27'
]
start_block = 15177000
end_block = 15178000
df = get_coinbase_trf(txn_list,start_block,end_block)
df

,0,1,2,3
0,15177999,0xf41775f2a03045511deee450845d6e197a2909ac46c2...,0.231752,0.013493
1,15177999,0xec7b3c821492dd76ac274b7ac37e0b55cabe535ecbd9...,0.517402,0.005718
